In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline

A. Importing and cleaning the data

You will start by downloading the data from the Kaggle link provided and import it into
pandas. You will then need to clean this data using the tools and methods covered in
this course. There is not just one correct way to do this. Several solutions will be
accepted as long as you demonstrate that you have thoroughly cleaned the data. This
includes, of course, dealing with missing values, normalizing entries and text data,
dealing with duplicates or incorrect values or getting certain data in more useful
formats.

In [2]:
#Import Data
#The full dataset has 356027 entries -> use dataset with 100'000
df = pd.read_csv('en.openfoodfacts.org.products.tsv',nrows=100000,low_memory=False,sep='\t')
print(df.info())
print()
print("Datatypes:")
print(df.dtypes.head(8))
print()
print(df.head(4))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 163 entries, code to water-hardness_100g
dtypes: float64(109), int64(3), object(51)
memory usage: 124.4+ MB
None

Datatypes:
code                       int64
url                       object
creator                   object
created_t                  int64
created_datetime          object
last_modified_t            int64
last_modified_datetime    object
product_name              object
dtype: object

    code                                                url  \
0   3087  http://world-en.openfoodfacts.org/product/0000...   
1   4530  http://world-en.openfoodfacts.org/product/0000...   
2   4559  http://world-en.openfoodfacts.org/product/0000...   
3  16087  http://world-en.openfoodfacts.org/product/0000...   

                      creator   created_t      created_datetime  \
0  openfoodfacts-contributors  1474103866  2016-09-17T09:17:46Z   
1             usda-ndb-import  1489069957  2017-03-09T14:32:

In [3]:
# missing values?
#-> many missing values!
df.isnull().sum().sum()

12314069

In [4]:
#Drop or fill columns ?
#-> too many missing
#Drop or fill makes no sense in general

dealing with duplicates

In [5]:
df.duplicated().sum()

0

In [6]:
# -> no duplicates

incorrect values: Detecting and filtering outliers

In [7]:
#Columns with Values:
dfNum=df.select_dtypes(include=['number'])
dfNum.columns.values

array(['code', 'created_t', 'last_modified_t', 'cities', 'allergens_en',
       'no_nutriments', 'additives_n', 'ingredients_from_palm_oil_n',
       'ingredients_from_palm_oil',
       'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil', 'nutrition_grade_uk',
       'energy_100g', 'energy-from-fat_100g', 'fat_100g',
       'saturated-fat_100g', '-butyric-acid_100g', '-caproic-acid_100g',
       '-caprylic-acid_100g', '-capric-acid_100g', '-lauric-acid_100g',
       '-myristic-acid_100g', '-palmitic-acid_100g', '-stearic-acid_100g',
       '-arachidic-acid_100g', '-behenic-acid_100g',
       '-lignoceric-acid_100g', '-cerotic-acid_100g',
       '-montanic-acid_100g', '-melissic-acid_100g',
       'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',
       'omega-3-fat_100g', '-alpha-linolenic-acid_100g',
       '-eicosapentaenoic-acid_100g', '-docosahexaenoic-acid_100g',
       'omega-6-fat_100g', '-linoleic-acid_100g', '-arachidonic-acid_100g',

In [8]:
#Filtering all columns with'_100g' 
d=df.dtypes
dd=d.filter(like='_100g', axis=0)
dd[:]
#df_new.iloc[:,dd[0]]

energy_100g                             float64
energy-from-fat_100g                    float64
fat_100g                                float64
saturated-fat_100g                      float64
-butyric-acid_100g                      float64
-caproic-acid_100g                      float64
-caprylic-acid_100g                     float64
-capric-acid_100g                       float64
-lauric-acid_100g                       float64
-myristic-acid_100g                     float64
-palmitic-acid_100g                     float64
-stearic-acid_100g                      float64
-arachidic-acid_100g                    float64
-behenic-acid_100g                      float64
-lignoceric-acid_100g                   float64
-cerotic-acid_100g                      float64
-montanic-acid_100g                     float64
-melissic-acid_100g                     float64
monounsaturated-fat_100g                float64
polyunsaturated-fat_100g                float64
omega-3-fat_100g                        

In [9]:
#Checking columns with'_100g' which are over max=100
dd=dfNum.loc[:,['salt_100g', 'zinc_100g','fat_100g','taurine_100g', 'magnesium_100g' ,'phosphorus_100g','calcium_100g','potassium_100g','folates_100g' ]]
dd.describe()

,salt_100g,zinc_100g,fat_100g,taurine_100g,magnesium_100g,phosphorus_100g,calcium_100g,potassium_100g,folates_100g
count,97056.000000,1921.000000,96968.000000,0.0,2504.000000,2808.000000,76106.000000,14933.000000,2057.000000
mean,2.148153,0.008353,11.982764,NaN,0.391378,0.248260,0.104927,0.297391,0.207088
std,15.727794,0.091355,17.430768,NaN,9.892177,0.261329,1.948243,0.890721,5.507824
min,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.068580,0.001280,0.000000,NaN,0.022750,0.088000,0.000000,0.104000,0.000048
50%,0.706120,0.004410,3.570000,NaN,0.070500,0.200000,0.033000,0.174000,0.000114
75%,1.597660,0.011720,18.920000,NaN,0.136000,0.357000,0.102750,0.318000,0.000214
max,2177.143220,4.000000,100.000000,NaN,364.286000,8.000000,522.727000,66.667000,178.571429


In [10]:
#-> Impossible values over 100g,  Filter it and delete it
#filter0 = dfNum.loc[:,['salt_100g']] > 100
filter0 = dd > 100
#filter0.head()

In [11]:
filter_any=filter0.any(axis=1)
#filter_any.head()

In [12]:
outliers = df.loc[filter_any, :]
#outliers.shape

In [13]:
df = df.drop(outliers.index, axis=0)
df.loc[:,['salt_100g', 'zinc_100g','fat_100g','taurine_100g', 'magnesium_100g' ,'phosphorus_100g','calcium_100g','potassium_100g','folates_100g' ]].describe()

,salt_100g,zinc_100g,fat_100g,taurine_100g,magnesium_100g,phosphorus_100g,calcium_100g,potassium_100g,folates_100g
count,96983.000000,1919.000000,96894.000000,0.0,2499.000000,2806.000000,76087.000000,14926.000000,2054.000000
mean,1.953079,0.008349,11.988706,NaN,0.112882,0.248231,0.098005,0.297188,0.039310
std,7.314303,0.091403,17.433321,NaN,0.766588,0.261337,0.454536,0.890581,1.165875
min,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.068580,0.001275,0.000000,NaN,0.022500,0.088000,0.000000,0.104000,0.000048
50%,0.706120,0.004410,3.570000,NaN,0.070000,0.200000,0.033000,0.174000,0.000114
75%,1.587500,0.011720,18.920000,NaN,0.136000,0.357000,0.102000,0.318000,0.000214
max,99.905820,4.000000,100.000000,NaN,37.500000,8.000000,78.000000,66.667000,38.460000


In [14]:
# check columns if empty
print(df.shape)
print(df.notnull().sum())

(99926, 163)
code                                    99926
url                                     99926
creator                                 99926
created_t                               99926
created_datetime                        99926
last_modified_t                         99926
last_modified_datetime                  99926
product_name                            99049
generic_name                              840
quantity                                 3000
packaging                                1758
packaging_tags                           1758
brands                                  98152
brands_tags                             98152
categories                               2843
categories_tags                          2843
categories_en                            2843
origins                                   349
origins_tags                              348
manufacturing_places                      650
manufacturing_places_tags                 650
labels               

In [15]:
# -> many columns are empty, therefore not used anymore -> delete
#Thresold min 100
df.dropna(thresh=100, axis=1,inplace=True)
print(df.notnull().sum())

code                         99926
url                          99926
creator                      99926
created_t                    99926
created_datetime             99926
last_modified_t              99926
last_modified_datetime       99926
product_name                 99049
generic_name                   840
quantity                      3000
packaging                     1758
packaging_tags                1758
brands                       98152
brands_tags                  98152
categories                    2843
categories_tags               2843
categories_en                 2843
origins                        349
origins_tags                   348
manufacturing_places           650
manufacturing_places_tags      650
labels                         858
labels_tags                    858
labels_en                      858
emb_codes                      106
emb_codes_tags                 105
purchase_places               1353
stores                         928
countries           